# Homework Assignment 7

### Topics: 

In [13]:
import boto3
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import dmc_cost_function
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
pd.set_option('display.max_columns', 50)

In [14]:
## 1. Using pandas to read the training and testing data files

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data-448-bucket-callaghan'
bucket = s3.Bucket(bucket_name)

file_key = 'train.csv'
file_key2 = 'test.csv'

bucket_object = bucket.Object(file_key)
bucket_object2 = bucket.Object(file_key2)

file_object = bucket_object.get()
file_object2 = bucket_object2.get()

file_content_stream = file_object.get('Body')
file_content_stream2 = file_object2.get('Body')

train = pd.read_csv(file_content_stream, sep = '|')
test = pd.read_csv(file_content_stream2, sep = '|')

train.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,0
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,0
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,0
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,0
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,0


In [15]:
## Creating variables from previous homework assignments

## Training set:

## Variable 1 (from decision tree)
train['Interaction_1'] = np.where((train['trustLevel'] <= 1.5) & (train['scannedLineItemsPerSecond'] <= 0.012) & 
                                  (train['lineItemVoids'] <= 10.5), 0, 1)
## Variable 2 (from decision tree)
train['Interaction_2'] = np.where((train['trustLevel'] <= 1.5) & (train['scannedLineItemsPerSecond'] > 0.012) & 
                                  (train['totalScanTimeInSeconds'] <= 895.0), 0, 1)
## Variable 3 (from decision tree)
train['Interaction_3'] = np.where((train['trustLevel'] > 1.5) & (train['grandTotal'] <= 99.145) & 
                                  train['trustLevel'] <= 2.5, 1, 0)
## Variable 4 (from decision tree)
train['Interaction_4'] = np.where((train['trustLevel'] > 1.5) & (train['grandTotal'] > 99.145) & 
                                  train['valuePerSecond'] <= 0.06, 1, 0)
## Variable 5 - Low trustLevel (all frauds came from trustLevel = 1 or 2)
train['lowTrust'] = np.where(train['trustLevel'] <= 2, 1, 0)

## Variable 6 - Made a quantity modification
train['madeModification'] = np.where(train['quantityModifications'] > 0, 1, 0)

## Variable 7 - Attempted a scan without registration
train['madeScansWithoutRegistration'] = np.where(train['scansWithoutRegistration'] > 0, 1, 0)

## Variable 8 - High or low totalScanTimeInSeconds (huge differnece in mean and median values for fraud/not fraud in this field)
train['lowTotalScanTime'] = np.where(train['totalScanTimeInSeconds'] < 1000, 1, 0)

## Varibales from strong heredity principle
train['Heredity_Feature_1'] = train['trustLevel'] * train['lowTrust']
train['Heredity_Feature_2'] = train['trustLevel'] * train['scannedLineItemsPerSecond']
train['Heredity_Feature_3'] = train['lowTrust'] * train['scannedLineItemsPerSecond']

## Heredity_Feature_3: all observations less than 0.012 are not fraud in this tree
train['New_Interaction_1'] = np.where(train['Heredity_Feature_3'] <= 0.012, 1, 0)

## Lots of positove observations when Heredity_Feature_3 > 0.012, totalScanTimeInSeconds > 1298.0, and trustLevel <= 1.5
train['New_Interaction_2'] = np.where((train['Heredity_Feature_3'] > 0.012) & (train['totalScanTimeInSeconds'] > 1298.0) 
                                      & (train['trustLevel'] < 1.5), 1, 0)

## Mostly all negative observations when Heredity_Feature_3 > 0.012, totalScanTimeInSeconds <= 1298.0, and Heredity_Feature_1 > 0.119
train['New_Interaction_3'] = np.where((train['Heredity_Feature_3'] > 0.012) & (train['totalScanTimeInSeconds'] <= 1298.0) 
                                      & (train['Heredity_Feature_1'] > 1.5), 1, 0)

In [16]:
## Testing set:

## Variable 1
test['Interaction_1'] = np.where((test['trustLevel'] <= 1.5) & (test['scannedLineItemsPerSecond'] <= 0.012) & 
                                  (test['lineItemVoids'] <= 10.5), 0, 1)
## Variable 2
test['Interaction_2'] = np.where((test['trustLevel'] <= 1.5) & (test['scannedLineItemsPerSecond'] > 0.012) & 
                                  (test['totalScanTimeInSeconds'] <= 895.0), 0, 1)
## Variable 3
test['Interaction_3'] = np.where((test['trustLevel'] > 1.5) & (test['grandTotal'] <= 99.145) & 
                                  test['trustLevel'] <= 2.5, 1, 0)
## Variable 4
test['Interaction_4'] = np.where((test['trustLevel'] > 1.5) & (test['grandTotal'] > 99.145) & 
                                  test['valuePerSecond'] <= 0.06, 1, 0)
## Variable 5 - Low trustLevel
test['lowTrust'] = np.where(test['trustLevel'] <= 2, 1, 0)

## Variable 6 - Made a quantity modification
test['madeModification'] = np.where(test['quantityModifications'] > 0, 1, 0)

## Variable 7 - Attempted a scan without registration
test['madeScansWithoutRegistration'] = np.where(test['scansWithoutRegistration'] > 0, 1, 0)

## Variable 8 - High or low totalScanTimeInSeconds
test['lowTotalScanTime'] = np.where(test['totalScanTimeInSeconds'] < 1000, 1, 0)

## Varibales from strong heredity principle
test['Heredity_Feature_1'] = test['trustLevel'] * test['lowTrust']
test['Heredity_Feature_2'] = test['trustLevel'] * test['scannedLineItemsPerSecond']
test['Heredity_Feature_3'] = test['lowTrust'] * test['scannedLineItemsPerSecond']

test['New_Interaction_1'] = np.where(test['Heredity_Feature_3'] <= 0.012, 1, 0)

test['New_Interaction_2'] = np.where((test['Heredity_Feature_3'] > 0.012) & (test['totalScanTimeInSeconds'] > 1298.0) 
                                      & (test['trustLevel'] < 1.5), 1, 0)

test['New_Interaction_3'] = np.where((test['Heredity_Feature_3'] > 0.012) & (test['totalScanTimeInSeconds'] <= 1298.0) 
                                      & (test['Heredity_Feature_1'] > 1.5), 1, 0)

In [17]:
## Defining the input and target variables
X_train = train.drop(columns = ['fraud'])
Y_train = train['fraud']

In [18]:
## 2.  Using the train data-frame (including the top 7 features from homework assignment 6) to do the following:

## (i) Considering a model to predict fraud:
## With the top 5 important features and using the GridSearchCV function with cv = 3 to run a hyper-parameter tuning procedure on the model
## With the top 6 important features and using the GridSearchCV function with cv = 3 to run a hyper-parameter tuning procedure on the model
## With the top 7 important features and using the GridSearchCV function with cv = 3 to run a hyper-parameter tuning procedure on the model

## Evaluating the models on the associated cost function from the Data Mining Cup task site

## Model: RandomForestClassifier

## Defining the parameter dictionary
rf_param_grid = {'n_estimators': [100, 300, 500], 'max_depth': [3, 5, 7], 'min_samples_split': [5, 10, 15], 
                  'min_samples_leaf': [5, 10, 15]}

## Defining the custom scorer
my_scorer = make_scorer(dmc_cost_function.cost_function, greater_is_better = True, needs_proba = True)


## -------------------------------

## Selecting the top-5 variables
X_train_sub = X_train[['trustLevel', 'New_Interaction_1', 'Heredity_Feature_3', 'totalScanTimeInSeconds', 'New_Interaction_2']]

## Running GridSearchCV with 3 folds
rf_grid_search_1 = GridSearchCV(RandomForestClassifier(), rf_param_grid, cv = 3, scoring = my_scorer, n_jobs = -1).fit(X_train_sub, Y_train)

## Extracting the best hyper-parameters
print('Best hyper-parameters for RandomForestClassifier with top-5 variables: \n', rf_grid_search_1.best_params_)
print('\nBest score:\n', rf_grid_search_1.best_score_)

## -------------------------------

## Selecting the top-6 variables
X_train_sub = X_train[['trustLevel', 'New_Interaction_1', 'Heredity_Feature_3', 'totalScanTimeInSeconds', 
                       'New_Interaction_2', 'valuePerSecond']]

## Running GridSearchCV with 3 folds
rf_grid_search_2 = GridSearchCV(RandomForestClassifier(), rf_param_grid, cv = 3, scoring = my_scorer, n_jobs = -1).fit(X_train_sub, Y_train)

## Extracting the best hyper-parameters
print('\nBest hyper-parameters for RandomForestClassifier with top-6 variables: \n', rf_grid_search_2.best_params_)
print('\nBest score:\n', rf_grid_search_2.best_score_)


## -------------------------------

## Selecting the top-7 variables
X_train_sub = X_train[['trustLevel', 'New_Interaction_1', 'Heredity_Feature_3', 'totalScanTimeInSeconds', 
                       'New_Interaction_2', 'valuePerSecond', 'New_Interaction_3']]

## Running GridSearchCV with 3 folds
rf_grid_search_3 = GridSearchCV(RandomForestClassifier(), rf_param_grid, cv = 3, scoring = my_scorer, n_jobs = -1).fit(X_train_sub, Y_train)

## Extracting the best hyper-parameters
print('\nBest hyper-parameters for RandomForestClassifier with top-7 variables: \n', rf_grid_search_3.best_params_)
print('\nBest score:\n', rf_grid_search_3.best_score_)

Best hyper-parameters for RandomForestClassifier with top-5 variables: 
 {'max_depth': 5, 'min_samples_leaf': 5, 'min_samples_split': 5, 'n_estimators': 500}

Best score:
 -20.0

Best hyper-parameters for RandomForestClassifier with top-6 variables: 
 {'max_depth': 7, 'min_samples_leaf': 5, 'min_samples_split': 5, 'n_estimators': 100}

Best score:
 -26.666666666666668

Best hyper-parameters for RandomForestClassifier with top-7 variables: 
 {'max_depth': 7, 'min_samples_leaf': 5, 'min_samples_split': 5, 'n_estimators': 500}

Best score:
 -20.0


In [19]:
## ii) Model: AdaBoostClassifier

## Defining the parameter dictionary
ada_param_grid = {'n_estimators': [100, 300, 500], 'base_estimator__min_samples_split': [5, 10, 15], 
                  'base_estimator__min_samples_leaf': [5, 10, 15], 'base_estimator__max_depth': [3, 5, 7], 
                  'learning_rate': [0.001, 0.01, 0.1, 1]}

## Defining the custom scorer
my_scorer = make_scorer(dmc_cost_function.cost_function, greater_is_better = True, needs_proba = True)


## -------------------------------

## Selecting the top-5 variables
X_train_sub = X_train[['trustLevel', 'New_Interaction_1', 'Heredity_Feature_3', 'totalScanTimeInSeconds', 'New_Interaction_2']]

## Running GridSearchCV with 3 folds
ada_grid_search_1 = GridSearchCV(AdaBoostClassifier(base_estimator = DecisionTreeClassifier()), ada_param_grid, 
                                 cv = 3, scoring = my_scorer, n_jobs = -1).fit(X_train_sub, Y_train)

## Extracting the best hyper-parameters
print('Best hyper-parameters for AdaBoostClassifier with top-5 variables: \n', ada_grid_search_1.best_params_)
print('\nBest score:\n', ada_grid_search_1.best_score_)

## -------------------------------

## Selecting the top-6 variables
X_train_sub = X_train[['trustLevel', 'New_Interaction_1', 'Heredity_Feature_3', 'totalScanTimeInSeconds', 
                       'New_Interaction_2', 'valuePerSecond']]

## Running GridSearchCV with 3 folds
ada_grid_search_2 = GridSearchCV(AdaBoostClassifier(base_estimator = DecisionTreeClassifier()), ada_param_grid, 
                                 cv = 3, scoring = my_scorer, n_jobs = -1).fit(X_train_sub, Y_train)

## Extracting the best hyper-parameters
print('\nBest hyper-parameters for AdaBoostClassifier with top-6 variables: \n', ada_grid_search_2.best_params_)
print('\nBest score:\n', ada_grid_search_2.best_score_)


## -------------------------------

## Selecting the top-7 variables
X_train_sub = X_train[['trustLevel', 'New_Interaction_1', 'Heredity_Feature_3', 'totalScanTimeInSeconds', 
                       'New_Interaction_2', 'valuePerSecond', 'New_Interaction_3']]

## Running GridSearchCV with 3 folds
ada_grid_search_3 = GridSearchCV(AdaBoostClassifier(base_estimator = DecisionTreeClassifier()), ada_param_grid, 
                                 cv = 3, scoring = my_scorer, n_jobs = -1).fit(X_train_sub, Y_train)

## Extracting the best hyper-parameters
print('\nBest hyper-parameters for AdaBoostClassifier with top-7 variables: \n', ada_grid_search_3.best_params_)
print('\nBest score:\n', ada_grid_search_3.best_score_)

Best hyper-parameters for AdaBoostClassifier with top-5 variables: 
 {'base_estimator__max_depth': 3, 'base_estimator__min_samples_leaf': 15, 'base_estimator__min_samples_split': 5, 'learning_rate': 0.01, 'n_estimators': 300}

Best score:
 -10.0

Best hyper-parameters for AdaBoostClassifier with top-6 variables: 
 {'base_estimator__max_depth': 7, 'base_estimator__min_samples_leaf': 10, 'base_estimator__min_samples_split': 10, 'learning_rate': 0.001, 'n_estimators': 500}

Best score:
 -3.3333333333333335

Best hyper-parameters for AdaBoostClassifier with top-7 variables: 
 {'base_estimator__max_depth': 5, 'base_estimator__min_samples_leaf': 5, 'base_estimator__min_samples_split': 15, 'learning_rate': 0.1, 'n_estimators': 100}

Best score:
 13.333333333333334


In [20]:
## iii) Model: Support Vector Classifier

## Defining the parameter dictionary
svc_param_grid = {'kernel': ['rbf', 'poly', 'sigmoid'], 'C': [0.01, 0.1, 1, 10], 'gamma': [0.01, 0.1, 1]}

## Defining the custom scorer
my_scorer = make_scorer(dmc_cost_function.cost_function, greater_is_better = True, needs_proba = True)


## -------------------------------

## Selecting the top-5 variables
scaler = MinMaxScaler(feature_range = (0, 1))
X_train_sub = scaler.fit_transform(X_train[['trustLevel', 'New_Interaction_1', 'Heredity_Feature_3', 'totalScanTimeInSeconds', 'New_Interaction_2']])

## Running GridSearchCV with 3 folds
svc_grid_search_1 = GridSearchCV(SVC(probability = True), svc_param_grid, cv = 3, scoring = my_scorer, n_jobs = -1).fit(X_train_sub, Y_train)

## Extracting the best hyper-parameters
print('Best hyper-parameters for AdaBoostClassifier with top-5 variables: \n', svc_grid_search_1.best_params_)
print('\nBest score:\n', svc_grid_search_1.best_score_)

## -------------------------------

## Selecting the top-6 variables
X_train_sub = scaler.fit_transform(X_train[['trustLevel', 'New_Interaction_1', 'Heredity_Feature_3', 
                                            'totalScanTimeInSeconds', 'New_Interaction_2', 'valuePerSecond']])

## Running GridSearchCV with 3 folds
svc_grid_search_2 = GridSearchCV(SVC(probability = True), svc_param_grid, cv = 3, scoring = my_scorer, n_jobs = -1).fit(X_train_sub, Y_train)

## Extracting the best hyper-parameters
print('\nBest hyper-parameters for AdaBoostClassifier with top-6 variables: \n', svc_grid_search_2.best_params_)
print('\nBest score:\n', svc_grid_search_2.best_score_)


## -------------------------------

## Selecting the top-7 variables
X_train_sub = scaler.fit_transform(X_train[['trustLevel', 'New_Interaction_1', 'Heredity_Feature_3', 
                                            'totalScanTimeInSeconds', 'New_Interaction_2', 'valuePerSecond', 'New_Interaction_3']])

## Running GridSearchCV with 3 folds
svc_grid_search_3 = GridSearchCV(SVC(probability = True), svc_param_grid, cv = 3, scoring = my_scorer, n_jobs = -1).fit(X_train_sub, Y_train)

## Extracting the best hyper-parameters
print('\nBest hyper-parameters for AdaBoostClassifier with top-7 variables: \n', svc_grid_search_3.best_params_)
print('\nBest score:\n', svc_grid_search_3.best_score_)

Best hyper-parameters for AdaBoostClassifier with top-5 variables: 
 {'C': 1, 'gamma': 1, 'kernel': 'poly'}

Best score:
 -83.33333333333333

Best hyper-parameters for AdaBoostClassifier with top-6 variables: 
 {'C': 0.1, 'gamma': 1, 'kernel': 'poly'}

Best score:
 -80.0

Best hyper-parameters for AdaBoostClassifier with top-7 variables: 
 {'C': 0.1, 'gamma': 1, 'kernel': 'poly'}

Best score:
 -81.66666666666667


In [ ]:
## From all of the above considered models, we see the best performance from the ______ model with the top-____ input variables and ______
## as hyper-parameters. This model has the highest cost function score across the grid search cross-validation process.